In [32]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

# User friendly names for columns
column_name_mapping = {
    "rec_num": "Record Number",
    "age_upon_outcome": "Age Upon Outcome",
    "animal_id": "Animal ID",
    "animal_type": "Animal Type",
    "breed": "Breed",
    "color": "Color",
    "date_of_birth": "Date of Birth",
    "datetime": "Date Time",
    "monthyear": "Month/Year",
    "name": "Name",
    "outcome_subtype": "Outcome Subtype",
    "outcome_type": "Outcome Type",
    "sex_upon_outcome": "Sex Upon Outcome",
    "location_lat": "Location Latitude",
    "location_long": "Location Longitude",
    "age_upon_outcome_in_weeks": "Age in Weeks Upon Outcome"
}


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Centering image and title for visual purposes
    html.Div(className="row", style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center'}, children=[
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width="171", height="171"),
        html.Div(style={'text-align': 'center'}, children=[
        html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
        html.H2("Austin Henley's SNHU CS-340 Dashboard"),
        ])
    ]),
    html.Hr(),
    html.Div([dcc.RadioItems(
        id='rescue-type',
        options=[
        # Creating radio buttons for filtering
        {'label':'Water Rescue', 'value': 'Water'},
        {'label':'Mountain Rescue', 'value': 'Mountain'},
        {'label':'Disaster Rescue', 'value': 'Disaster'},
        {'label': 'Reset Data', 'value': 'all'}],
        inline=True)
        ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": column_name_mapping.get(i, i), "id": i, "deletable": False, "selectable": True} for i in df.columns],
        # Converting dataframe to dictionary
        data=df.to_dict('records'),
        # Cells cannot be edited
        editable=False,
        # Enabling sorting on the table using browser itself
        sort_action="native",
        # Allowing multi-column sorting
        sort_mode="multi",
        # Allow selectable rows
        row_selectable = "single",
        # Sets default selected row to the first (0)
        selected_rows=[0],
        # Enables paging
        page_action="native",
        # Sets default page to the first (0)
        page_current=0,
        # Sets how many records per page
        page_size=10,
        # Enables scroll bar for viewing all columns
        style_table={'overflowX': 'scroll'}),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('rescue-type', 'value')])

def update_dashboard(rescue_type):
    # Filter and query for water rescue dogs
    if rescue_type == 'Water':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
        }
        
    # Filter and query for mountain rescue dogs
    elif rescue_type == 'Mountain':
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
        }
    
    # Filter and query for disaster rescue dogs
    elif rescue_type == 'Disaster':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20.0, '$lte': 300.0}
        }
        
    # Reset to full dataset
    else:
        query = {}
        
    # Creating filtered dataset and removing id column
    df_filtered = pd.DataFrame.from_records(db.read(query))
    df_filtered.drop(columns=['_id'],inplace=True)
    
    columns=[{"name": column_name_mapping.get(i, i), "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df_filtered.to_dict('records')
            
    return (data, columns)
      
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Do not update if data is None
    if viewData is None:
        return
    
    # Preferred breeds based of documentation
    preferred_breeds = [
    'Labrador Retriever Mix', 
    'Chesapeake Bay Retriever', 
    'Newfoundland', 
    'German Shepherd', 
    'Alaskan Malamute', 
    'Old English Sheepdog', 
    'Siberian Husky', 
    'Rottweiler', 
    'Doberman Pinscher', 
    'Golden Retriever', 
    'Bloodhound'
    ]

    # Dataset for the pie graph
    dffPie = pd.DataFrame.from_dict(viewData)
    
    # Filter for only the preferred breeds
    dffPie = dffPie[dffPie['breed'].isin(preferred_breeds)]
    
    # Creating Pie graph and fixing layout to show centered title
    figure = px.pie(dffPie, names='breed')
    figure.update_layout(
        title=dict(text='Preferred Breed Percentages - Dogs', font=dict(size=20), x = 0.5),
        margin=dict(t=150),
        title_y=0.95
    )
    
    # Return graph to dashboard
    return [
        dcc.Graph(figure=figure)
    ]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    # Data frame for map 
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:12258/
